# Recurrent Neural Networks

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Ongoing&color=orange)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/inefficient-networks/blob/master/docs/notebooks/tensorflow/05-tensorflow-cnn.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/inefficient-networks?style=social)](https://github.com/particle1331/inefficient-networks)

---

## Introduction

Recall that the idea for using convolutions is that close pixels are related. But pixels far from each other have none. For sequences, this is not necessarily true. There can be long term dependencies between words in a at the beginning and end of a paragraph, for example. For modelling sequences, we will look at **recurrent connections** (i.e. cyclic dependencies) and its extensions. As with convolutional layers, recurrent units also use weight sharing but over time instead of over space.

The key idea for sequence modelling is that while sequences $\langle \boldsymbol{\mathsf{x}}_1, \boldsymbol{\mathsf{x}}_2, \ldots, \boldsymbol{\mathsf{x}}_T \rangle$ have arbitrary length $T \in \mathbb{N},$ each time step can be modelled with a state vector $\boldsymbol{\mathsf{x}}_t \in \mathbb{R}^d$ with fixed number of entries. For example, when modelling temperatures for each day, we can use a sequence of maximum, minimum, and average temperatures getting a 3-dimensional vector to represent the state of a day. For this to work well, each step in a sequence must be semantically equivalent, and that order matters. Recurrent connections are able to capture this information in the data by using a memory vector $\boldsymbol{\mathsf{h}}_t.$

In [ ]:
import warnings
from pathlib import Path

import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_inline import backend_inline

import tensorflow.keras as kr
import tensorflow as tf

DATASET_DIR = Path("./data").absolute()
RANDOM_SEED = 42

warnings.simplefilter(action="ignore")
backend_inline.set_matplotlib_formats('svg')

## Recurrent connections

Recurrent connections compute a hidden state vector $\boldsymbol{\mathsf{h}}_t$ which evolves based on the new system state $\boldsymbol{\mathsf{x}}_t$ and the existing hidden state $\boldsymbol{\mathsf{h}}_{t-1}.$ This has weights both for blending the past history to the current state of the system:

$$
\begin{aligned}
\boldsymbol{\mathsf{h}}_t 
&= \textsf{A}(\boldsymbol{\mathsf{h}}_{t-1}, \boldsymbol{\mathsf{x}}_{t}) \\
&= \tanh\left( \boldsymbol{\mathsf{h}}_{t-1} \boldsymbol{\mathsf{W}}_{\mathsf{h}} +  {\boldsymbol{\mathsf{x}}}_t\, \boldsymbol{\mathsf{W}}_{\mathsf{x}}  + \boldsymbol{\mathsf{b}}\right)
\end{aligned}
$$

such that $\boldsymbol{\mathsf{h}}_0 = \boldsymbol 0.$ Note that this is able to process an entire sequence regardless of its length. The choice of nonlinearity means that components of the hidden state saturate in the range $[-1, 1].$ 

Unrolling recurrent connections makes it look more familiar:

```{margin}
Source:<br>
[`d2l.ai/ch9`](https://www.d2l.ai/chapter_recurrent-neural-networks/index.html)
```
```{figure} ../../img/unfolded-rnn.svg
---
width: 80%
name: unfolded-rnn
---
```

**Example.** Input order matters:

In [ ]:
rnn = kr.layers.SimpleRNN(3, input_shape=(None, 2))
x = tf.random.normal((1, 10, 2))
rnn(x)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-19 23:07:54.065335: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-19 23:07:54.065572: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.9656761 , -0.363641  , -0.94827056]], dtype=float32)>

In [ ]:
Wy, Wx, b = rnn.weights

In [ ]:
Wy

<tf.Variable 'simple_rnn/simple_rnn_cell/kernel:0' shape=(2, 3) dtype=float32, numpy=
array([[ 0.2618786 , -0.94552475,  0.33985174],
       [-0.9222554 , -0.19650507, -0.38553113]], dtype=float32)>

In [ ]:
T = 3
x  = np.random.random((T, 3))
Wx = np.random.random((3, 2))
Wh = np.random.random((2, 2))


def run(order=[0, 1, 2]):
    h  = np.zeros((1, 2))

    print('\nh=')
    ht = []
    for i in order:
        h = np.tanh(h @ Wh + x[[i]] @ Wx)
        ht.append(h.sum())
    return ht

print(run([0, 1, 2]))
print(run([1, 0, 2]))


h=
[1.298508403153427, 1.4735615325954123, 1.6886432937887488]

h=
[0.6852771463365646, 1.564896581198873, 1.700658938689842]


## Backpropagation Through Time (BPTT)

## PyTorch implementation

In this section, we will look at how to use RNN units in PyTorch to predict the likely country of origin of a name. The code for this section is based on [this notebook](https://github.com/EdwardRaff/Inside-Deep-Learning/blob/main/Chapter_4.ipynb). For this task we will classify a name's source language by passing the characters of a name as sequence that is fed into the RNN unit. Each character updates the hidden state vector. Once the complete name has been processed, we get a final state vector, which is passed to a classification subnetwork for prediction.

```{margin}
Source:<br>

```
```{figure} ../../img/rnn-names.png
---
width: 500px
name: rnn-names
---

Classifying the source language for the name Frank. The characters of the name is sequentially passed to an RNN unit resulting in a final hidden state $\boldsymbol{\mathsf{h}}_5$ that is passed to  a linear layer.
```

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

device = torch.device('mps')

Downloading the dataset:

In [ ]:
import requests, zipfile, io

file_url = "https://download.pytorch.org/tutorial/data.zip"
zip = zipfile.ZipFile(io.BytesIO(requests.get(file_url).content))
zip.extractall()

Removing unicode (e.g. Ślusàrski to Slusarski):

In [ ]:
import unicodedata
import string

# https://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    alphabet = {}
    for i, a in enumerate(string.ascii_letters + " .,;'"):
        alphabet[a] = i

    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn' and c in alphabet
    )

# Loop through every language:
data = {}
for path in (p for p in zip.namelist() if "names" in p and p.endswith(".txt")):
    lang = path.split('/')[-1].replace(".txt", "")
    with zip.open(path) as f:
        lang_names = [unicode_to_ascii(line).lower() for line in str(f.read(), encoding='utf-8').strip().split("\n")]
        data[lang] = lang_names
    
for lang, lang_names in data.items():
    print(f"{lang+':':<12} {len(lang_names):>12}")

Arabic:              2000
Chinese:              268
Czech:                519
Dutch:                297
English:             3668
French:               277
German:               724
Greek:                203
Irish:                232
Italian:              709
Japanese:             991
Korean:                94
Polish:               139
Portuguese:            74
Russian:             9408
Scottish:             100
Spanish:              298
Vietnamese:            73
